In [1]:
import sys
import pandas as pd
import sqlite3
import glob
from sqlalchemy import create_engine
conn = sqlite3.connect('datenbank.db')
cursor = conn.cursor()

In [2]:
def update_database():
    try:
        # Analytik-Daten aus Messsystem-Tabelle ablesen
        files = glob.glob('O:/Datenmanagement/Befunde und Messsysteme/Messsysteme/1_Auftragslabore\*.xlsx*')
        for i in range(len(files)):
            if 'Messsysteme und Preise aller Auftragslabore' in files[i]:
                path = files[i]
                break
        xls = pd.ExcelFile(path)
        # Alle Reiter in Liste wiedergeben
        sheet_names = xls.sheet_names
        sheet_names.remove('Tabelle1')
        # Alle Tabellen von Laboren ablesen
        tables = []
        for sheet in sheet_names:
            df = pd.read_excel(path, sheet_name = sheet)
            tables.append(df)

        # Tabellen richtig ordnen
        # bis auf LADR sind alle Tabellen gleich strukturiert
        for i in range(len(tables)):
            if i != 6:
                tables[i].columns = tables[i].iloc[1].tolist()
                tables[i] = tables[i][['Parameter', 'Angebots- Preis \n(excl. MwSt)', 
                                       'Methode', 'Gerät', 'Hersteller/ Gerät']]
                tables[i].columns = ['Parameter', 'Preis', 'Methode', 'Messsystem', 'Hersteller']
                tables[i] = tables[i].iloc[2:]
            else:
                tables[i].columns = tables[i].iloc[2].tolist()
                tables[i] = tables[i][['laboratory parameter', 'Angebots- Preis \n(excl. MwSt)',
                                       'measuring method', 'measuring System', 'manufacturer of measuring system']]
                tables[i].columns = ['Parameter', 'Preis', 'Methode', 'Messsystem', 'Hersteller']
                tables[i] = tables[i].iloc[3:]

        # Liste aller Parameter in Datenbank
        query = "SELECT name FROM namen"
        engine = create_engine("sqlite:///datenbank.db")
        parameter = pd.read_sql(query, engine)
        parameter_liste = list(parameter['name'])

        # Tabelle analytik leeren
        query = "DELETE FROM analytik;"
        cursor.execute(query)

        # Suche in der ersten Spalte der Tabellen nach Parameter
        # Identifizieren des Labors
        for parameter in parameter_liste:
            labor = []
            for i in range(len(tables)):
                for j in range(len(tables[i])):
                    if parameter in str(tables[i].iloc[j,0]):
                        labor.append(sheet_names[i])
                        break

            # Ablesen der Messsysteme, Tests etc.
            for lab in labor:
                table = tables[sheet_names.index(lab)]
                parameter_rows = []
                for i in range(len(table)):
                    if parameter in table.iloc[i,0]:
                        parameter_rows.append(i)
                table_parameter = table.iloc[parameter_rows,1:]
                table_parameter = table_parameter.reset_index().drop('index', axis=1)
                # Eintrag in Datenbank
                for i in range(len(table_parameter)):
                    meth = table_parameter.iloc[i,1]
                    messsys = table_parameter.iloc[i,2]
                    herst = table_parameter.iloc[i,3]
                    try:
                        pr = round(table_parameter.iloc[i,0], 2)
                    except:
                        pr = table_parameter.iloc[i,0]
                    query = '''INSERT INTO analytik (name, labor, methode, messsystem, hersteller, preis)
                               VALUES (?, ?, ?, ?, ?, ?);'''
                    cursor.execute(query, (parameter, lab, meth, messsys, herst, pr))

        conn.commit()
        return True
    except:
        return False
    
if __name__ == "__main__":
    if update_database():
        sys.exit(0)  # Return 0 for success
    else:
        sys.exit(1)  # Return non-zero value for failure

SystemExit: 0

C:\Users\l.nguyen quoc\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3468: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
